<a href="https://colab.research.google.com/github/fervenceslau/ravendawn/blob/main/CaveMapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 as cv
import numpy as np
import sys
from tqdm.notebook import tqdm

def CropImage(image, scale):
    """
    Crops a region inside a 3-channel image relative to its dimensions through a scaling factor.
    """
    scaleX, scaleY = [scale, scale]
    if (type(scale) == list):
        scaleX, scaleY = scale
    width  = image.shape[1]
    height = image.shape[0]
    left   = int(0.5 * (1 - scaleX) * width)
    right  = int(left + width * scaleX)
    top    = int(0.5 * (1 - scaleY) * height)
    bottom = int(top + height * scaleY)
    return image[top:bottom, left:right, :]

def RemovePlayer(image):
    """
    Fills a black rectangle inside a 3-channel image position where the player should be.
    """
    return cv.rectangle(image, (850, 426), (1006, 558), (0, 0 ,0), -1)

def ResizeImage(image, scale):
    """
    Resizes an image with a global scaling factor.
    """
    width  = int(image.shape[1] * scale)
    height = int(image.shape[0] * scale)
    dim = (width, height)
    return cv.resize(image, dim)

def SelectRegionForFeatures(image, scale):
    """
    Selects a region inside a 3-channel image to find tracking features.
    The region is a cropped rectangle around the player without the player.
    """
    image = RemovePlayer(image)
    image = CropImage(image, scale)
    return image

def DetectAndDescribe(image):
    """
    Detects and returns keypoints and features used to track movement between images.
    """
    descriptor = cv.BRISK_create(thresh=10)
    (kps, features) = descriptor.detectAndCompute(image, None)
    return (kps, features)

def MatchKeyPointsBF(featuresA, featuresB):
    """
    Returns matched features using a brute-force descriptor matcher.
    """
    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
    best_matches = bf.match(featuresA, featuresB)
    rawMatches = sorted(best_matches, key = lambda x:x.distance)
    return rawMatches

def EstimateMovement(kpsA, kpsB, matches):
    """
    Estimates movement based on matching keypoints and estimateAffinePartial2D.
    The movement is the last column of the affine matrix [R p], but since the estimation process
    can be faulty, first we remove the influence of the rotation matrix [R], to get [I R^{-1}p].
    """
    kpsA = np.float32([kp.pt for kp in kpsA])
    kpsB = np.float32([kp.pt for kp in kpsB])
    if len(matches) > 4:
        ptsA = np.float32([kpsA[m.queryIdx] for m in matches])
        ptsB = np.float32([kpsB[m.trainIdx] for m in matches])
        (affineMatrix, inliers) = cv.estimateAffinePartial2D (ptsA, ptsB)
        movement = np.matmul(np.linalg.inv(affineMatrix[:, 0:2]), affineMatrix[:, 2])
        return movement
    else:
        return None
    
def CalculateImageLimits(baseShape, newShape, movement):
    """
    Calculates the output image limits after applying a movement to a new image.
    With these limits, calculate the image dimensions and offset to make the minimum coordinates (0, 0) once again.
    """
    baseLimits = np.array([[0, 0], np.flip(baseShape[0:-1]) - [1, 1]]).T
    newLimits  = np.array([[0, 0], np.flip(newShape[0:-1]) - [1, 1]]).T + np.tile(movement, (2, 1)).T
    limits     = np.concatenate((baseLimits, newLimits), axis=1)
    limits     = np.vstack((limits.min(axis=1), limits.max(axis=1))).T
    offset     = -limits[:, 0]
    dimension  = (limits[:, 1] + offset + [1, 1]).astype(np.int32)
    return limits, offset, dimension

def OffsetImage(image, offset, dimension):
    """
    Applies an offset to a given image and make the resulting image have the desired dimension.
    """
    offset = offset.astype(np.int32)
    result = np.zeros(np.hstack((np.flip(dimension), 3))).astype(np.uint8)
    result[offset[1]:(offset[1] + image.shape[0]),
           offset[0]:(offset[0] + image.shape[1]),
           :] = image
    return result

def StitchImages(imageA, imageB, movement, offset, dimension):
    """
    Stitch two images together based on the relative movement. 
    The images are blended together using max() function to keep high brightness values.
    The output image will have the desired dimensions and the offset variable is used to correct the minimum
    limit of the resulting image to make it (0, 0).
    """
    auxA = OffsetImage(imageA, offset, dimension)
    auxB = OffsetImage(imageB, offset + movement, dimension)
    result = np.maximum(auxA, auxB)
    return result.astype(np.uint8)

In [ ]:
!git clone https://github.com/fervenceslau/ravendawn.git

# Create a VideoCapture object and check if file was opened successfully
cap = cv.VideoCapture('sample.mp4')
if (cap.isOpened()== False): 
  print("Error opening video file")
  sys.exit()

# Get information from video file
height     = cap.get(cv.CAP_PROP_FRAME_HEIGHT)
width      = cap.get(cv.CAP_PROP_FRAME_WIDTH) 
fps        = cap.get(cv.CAP_PROP_FPS)
frameCount = cap.get(cv.CAP_PROP_FRAME_COUNT)
duration   = frameCount / fps

# Define video times to use in the algorithm
videoTimeStart = 5
videoTimeStop  = 0    # TODO: add stop time functionality
videoTimeStep  = 0.1

# Frame information used to read frames from the VideoCapture
frameIdx    = 0
frameStep   = videoTimeStep * fps
frameStart  = videoTimeStart * fps
frameNumber = int(frameStart + frameIdx * frameStep)
frameTotal  = int((frameCount - frameNumber) / frameStep)

# Define variables used in the stitching algorithm
images = [[], []]
result = []
offset = np.array([0, 0])
movement = np.array([0, 0])

# Read frames from the VideoCapture and perform image stitching (using a progress bar)
for frameIdx in tqdm(range(frameTotal)):
    
    # Read a frame
    ret = False
    while (not ret):
        cap.set(1, frameNumber);   
        ret, frame = cap.read()
        frame = frame[1:-64, 1:-1, :] # Removes black outline from screenshot

    # Update images list
    images[0] = images[1]
    images[1] = frame

    # Update sample number
    frameNumber = int(frameStart + frameIdx * frameStep)

    # Stich images after acquiring two images
    if (frameIdx > 1):

        # Select image regions used to track movement
        imgA = SelectRegionForFeatures(images[0], [0.5, 0.5])
        imgB = SelectRegionForFeatures(images[1], [0.5, 0.5])

        # Obtain tracking keypoints and features
        kpsA, featuresA = DetectAndDescribe(imgA)
        kpsB, featuresB = DetectAndDescribe(imgB)

        # Find matches between tracking points
        matches = MatchKeyPointsBF(featuresA, featuresB)

        # Initializes resulting stitched image image
        if (len(result) == 0):
            result = images[0]

        # Calculate overall new frame movement and compensate for the previous offset
        # The estimated movement is rounded to avoid error accumulation caused by subpixel movements...
        movement = movement + -np.round(EstimateMovement(kpsA, kpsB, matches))
        movement = movement + offset

        # Calculate the new image limits, offset and dimension
        limits, offset, dimension = CalculateImageLimits(result.shape, images[1].shape, movement)

        # Stitch new frame to the results
        result = StitchImages(result, images[1], movement, offset, dimension)

        # Write the result to an output file every N frames
        if (frameIdx % 50 == 0):
            cv.imwrite('result.png', result)

        # Display the results for each step
#         cv.imshow("", result)
#         if cv.waitKey() & 0xFF == ord('q'):
#             break

# Write the result to an output file after all code execution
cv.imwrite('result.png', result)
    
# Releases VideoCapture and close all cv windows
cap.release()
cv.destroyAllWindows()

Error opening video file


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
%ls

sample_data/
